In [1]:
from pathlib import Path
import numpy as np
import torch
import matplotlib.pyplot as plt
from natsort import natsorted

from pathlib import Path
import cv2
import numpy as np
from tqdm import tqdm

# reload notebook automatically after changes to source python files
%load_ext autoreload
%autoreload 2

# change base folder to parent
import os
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
print(os.getcwd())

# for evo ######################################################################
from evo.tools import log
log.configure_logging(verbose=False, debug=False, silent=False)

import pprint
import numpy as np

from evo.tools import plot
import matplotlib.pyplot as plt

# temporarily override some package settings
from evo.tools.settings import SETTINGS
SETTINGS.plot_usetex = False

plot.apply_settings(SETTINGS)
%matplotlib inline

from evo.tools import file_interface
#############################################################################

/home/onyx/work_dirs/rjayanti/mast3r-vloc


### Write Poses Camera TUM

In [2]:
from src.datasets.dataset_utils import load_tum_poses, PoseMode
from src.utils.tf_utils import compose_qt_tf, decompose_tf_qt

dataset_root = Path("/home/onyx/work_dirs/rjayanti/mast3r-vloc/data/TorWIC-SLAM/")
ref_scene_folder = dataset_root / "Jun23" / "Aisle_CW_Run_2"

ref_poses_file = ref_scene_folder / "traj_gt.txt"
timestamps, ref_poses = load_tum_poses(ref_poses_file, PoseMode.MAT4x4)

qvec_l2c_right = np.array([0.41406507, -0.6100328, 0.57049433, -0.3618651])
tvec_l2c_right = np.array([0.07686256, -0.15441064, -0.1026438])
tf_l2c_right = compose_qt_tf(qvec_l2c_right, tvec_l2c_right, in_xyzw=True)

qvec_l2c_left = np.array([-0.6116725, 0.39292797, -0.3567415, 0.58668551])
tvec_l2c_left = np.array([0.12944592, 0.04299934, -0.1137434])
tf_l2c_left = compose_qt_tf(qvec_l2c_left, tvec_l2c_left, in_xyzw=True)

poses_camera_right_tum = []
poses_camera_left_tum = []
for i, tf_w2l in enumerate(ref_poses):
    tf_w2c_right = tf_w2l @ tf_l2c_right
    tf_w2c_left = tf_w2l @ tf_l2c_left
    qvec_w2c_right, tvec_w2c_right = decompose_tf_qt(tf_w2c_right, in_xyzw=True)
    qvec_w2c_left, tvec_w2c_left = decompose_tf_qt(tf_w2c_left, in_xyzw=True)
    
    # build TUM format pose as [timestamp] [tx] [ty] [tz] [qx] [qy] [qz] [qw]
    pose_camera_right_tum = np.concatenate([np.array([i]), tvec_w2c_right, qvec_w2c_right])
    pose_camera_left_tum = np.concatenate([np.array([i]), tvec_w2c_left, qvec_w2c_left])
    
    poses_camera_right_tum.append(pose_camera_right_tum)
    poses_camera_left_tum.append(pose_camera_left_tum)
    
poses_camera_right_tum = np.array(poses_camera_right_tum)
poses_camera_left_tum = np.array(poses_camera_left_tum)
# write poses to file
poses_camera_right_tum_file = ref_scene_folder / "poses_camera_right_tum.txt"
poses_camera_left_tum_file = ref_scene_folder / "poses_camera_left_tum.txt"
np.savetxt(poses_camera_right_tum_file, poses_camera_right_tum, fmt='%.9f')
np.savetxt(poses_camera_left_tum_file, poses_camera_left_tum, fmt='%.9f')